# Business Problem

For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful and will provide recommendations to the stakeholder on how to make a successful movie.

Over the course of this project, you will:

- Part 1: Download several files from IMDB’s movie data set and filter out the subset of movies requested by the stakeholder.
- Part 2: Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.
- Part 3: Construct and export a MySQL database using your data.
- Part 4: Apply hypothesis testing to explore what makes a movie successful.
- Part 5 (Optional): Produce a Linear Regression model to predict movie performance.

For Part 1 of the project, you will be creating your project repository, loading the official IMDB data for the requested tables, filtering out unnecessary data, and saving the filtered tables as gzip-compressed csv files (".csv.gz") in your repository.

## The Data
- IMDB Provides Several Files with varied information for Movies, TV Shows, Made for TV Movies, etc.

 - Overview/Data Dictionary: https://www.imdb.com/interfaces/
 - Downloads page: https://datasets.imdbws.com/


- From their previous research, they realized they want to focus on the following files:

 - title.basics.tsv.gz
 - title.ratings.tsv.gz
 - title.akas.tsv.gz

## Specifications

Your stakeholder only wants you to include information for movies based on the following specifications:

- Exclude any movie with missing values for genre or runtime
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre)
- Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- Include only movies that were released in the United States

## Deliverable
After filtering out movies that do not meet the stakeholder's specifications:

- Before saving, run a final .info() for each of the dataframes to show a summary of how many movies remain and the datatypes of each feature
- Save each file to a compressed csv file "Data/" folder inside your repository.
- Commit your changes to your repository in GitHub desktop and Publish repository / Push Changes.
- Submit the link to your repository

# Part 1: Download files from IMDB’s movie data set

## Imports

In [1]:
import pandas as pd
import numpy as np

#pd.set_option('display.max_rows', None) 

## Load Data

In [2]:
# set urls for database from IMDB website
url_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz'
url_akas = 'https://datasets.imdbws.com/title.akas.tsv.gz'
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [3]:
# load data
basics = pd.read_csv(url_basics, sep = '\t', low_memory = False)
akas = pd.read_csv(url_akas, sep = '\t', low_memory = False)
ratings = pd.read_csv(url_ratings, sep = '\t', low_memory = False)

## Data Cleaning

### Title Basics Database

#### Replace "\N" with np.nan

In [4]:
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            15
dtype: int64

In [5]:
# Missing values are nan and \N. I wlll replace them all with nan so I can delete them. 
basics.replace({'\\N':np.nan}, inplace = True)
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1335303
endYear           9790326
runtimeMinutes    6973172
genres             444842
dtype: int64

#### Eliminate movies that are null for runtimeMinutes

In [6]:
basics.dropna(subset = ['runtimeMinutes'], axis = 0, inplace = True)
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 1
startYear          171428
endYear           2872764
runtimeMinutes          0
genres              77280
dtype: int64

#### Eliminate movies that are null for genre

In [7]:
basics.dropna(subset = ['genres'], axis = 0, inplace = True)
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          166487
endYear           2797079
runtimeMinutes          0
genres                  0
dtype: int64

#### Keep only titleType==Movie

In [8]:
basics = basics[basics['titleType'] == 'movie']
basics['titleType'].info()

<class 'pandas.core.series.Series'>
Int64Index: 383323 entries, 8 to 9897625
Series name: titleType
Non-Null Count   Dtype 
--------------   ----- 
383323 non-null  object
dtypes: object(1)
memory usage: 5.8+ MB


In [9]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383323 entries, 8 to 9897625
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          383323 non-null  object
 1   titleType       383323 non-null  object
 2   primaryTitle    383323 non-null  object
 3   originalTitle   383323 non-null  object
 4   isAdult         383323 non-null  object
 5   startYear       376863 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  383323 non-null  object
 8   genres          383323 non-null  object
dtypes: object(9)
memory usage: 29.2+ MB


#### Keep startYear 2000-2022

In [10]:
basics.dropna(subset = ['startYear'], axis = 0, inplace = True)
basics['startYear'] = basics['startYear'].astype(dtype = int) 
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]
basics['startYear'].describe()

count    223617.000000
mean       2013.373518
std           5.853839
min        2000.000000
25%        2009.000000
50%        2014.000000
75%        2018.000000
max        2022.000000
Name: startYear, dtype: float64

#### Eliminate movies that include "Documentary" in genre

In [11]:
basics['genres'].value_counts()

Documentary                  53333
Drama                        36068
Comedy                       13458
Comedy,Drama                  6461
Horror                        5806
                             ...  
Crime,Documentary,Romance        1
Animation,Biography,Sport        1
Adventure,History,Music          1
Adventure,History,War            1
Crime,Fantasy,Sci-Fi             1
Name: genres, Length: 1187, dtype: int64

In [12]:
is_documentary = basics['genres'].str.contains('documentary',case = False)
basics = basics[~is_documentary]

In [13]:
basics['genres'].value_counts()

Drama                     36068
Comedy                    13458
Comedy,Drama               6461
Horror                     5806
Drama,Romance              4318
                          ...  
Action,Fantasy,Western        1
Family,Musical,Sport          1
Horror,Music,Mystery          1
Comedy,History,Mystery        1
Crime,Fantasy,Sci-Fi          1
Name: genres, Length: 966, dtype: int64

### AKAS Database

In [14]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36084220 entries, 0 to 36084219
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


#### Keep only US movies.

In [15]:
akas = akas[akas['region'] == 'US']
akas['region'].value_counts()

US    1440940
Name: region, dtype: int64

#### Replace "\N" with np.nan

In [16]:
akas.replace({'\\N':np.nan}, inplace = True)
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1437000
types               461368
attributes         1394264
isOriginalTitle       1345
dtype: int64

#### Keep only US movies using AKAs table

In [17]:
keepers =basics['tconst'].isin(akas['titleId'])
basics = basics[keepers]

### RATINGS Database

In [18]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1317803 entries, 0 to 1317802
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1317803 non-null  object 
 1   averageRating  1317803 non-null  float64
 2   numVotes       1317803 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.2+ MB


#### Replace "\N" with np.nan (if any)

In [19]:
ratings.replace({'\\N':np.nan}, inplace = True)
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

#### Keep only US movies from Title Basics Table using AKAs table

In [20]:
keepers =ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers]

## Review

In [21]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86790 entries, 34803 to 9897441
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          86790 non-null  object
 1   titleType       86790 non-null  object
 2   primaryTitle    86790 non-null  object
 3   originalTitle   86790 non-null  object
 4   isAdult         86790 non-null  object
 5   startYear       86790 non-null  int64 
 6   endYear         0 non-null      object
 7   runtimeMinutes  86790 non-null  object
 8   genres          86790 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.6+ MB


In [22]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1440940 entries, 5 to 36083964
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1440940 non-null  object
 1   ordering         1440940 non-null  int64 
 2   title            1440940 non-null  object
 3   region           1440940 non-null  object
 4   language         3940 non-null     object
 5   types            979572 non-null   object
 6   attributes       46676 non-null    object
 7   isOriginalTitle  1439595 non-null  object
dtypes: int64(1), object(7)
memory usage: 98.9+ MB


In [23]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500581 entries, 0 to 1317778
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         500581 non-null  object 
 1   averageRating  500581 non-null  float64
 2   numVotes       500581 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.3+ MB


## Save Files

In [24]:
import os
os.makedirs('Data/',exist_ok=True) 

# Confirm folder created
os.listdir("Data/")

['.DS_Store',
 'title_basics.csv.gz',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

In [25]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [26]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [27]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1976
1,tt0000002,5.8,264
2,tt0000005,6.2,2619
3,tt0000006,5.1,182
4,tt0000007,5.4,821


# Part 2: Use an API to extract  revenue and profit

## Your Stakeholder Wants More Data!
After investigating the preview of your data from Part 1, your stakeholder realized that there is no financial information included in the IMDB data (e.g. budget or revenue).

This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before you will be able to determine which movies are successful.
Your stakeholder identified The Movie Database ([TMDB](https://www.themoviedb.org/)) as a great source of financial data . Thankfully, TMDB offers a free API for programmatic access to their data!

Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

- Note: this process can take a long time and may need to run overnight.

### Specifications - Financial Data
Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

Each year should be saved as a separate .csv.gz file

Confirm Your API Function works.

- To ensure your function for extracting movie data from TMDB is working, test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that your function runs without error and that it returns the correct movie's data for both test ids.

Once you have retrieved and saved the final results to 2 separate .csv.gz files, move on to a new Exploratory Data Analysis notebook to explore the following questions.

### Exploratory Data Analysis
- Load in your csv.gz's of results for each year extracted.
- Concatenate the data into 1 dataframe for the remainder of the analysis.
- Once you have your data from the API, they would like you to perform some light EDA to show:
- How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
- Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.
- How many movies are there in each of the certification categories (G/PG/PG-13/R)?
- What is the average revenue per certification category?
- What is the average budget per certification category?

### Deliverables
After you have joined the tmdb results into 1 dataframe in the EDA Notebook:

- Save a final merged .csv.gz of all of the tmdb api data
- The file name is "tmdb_results_combined.csv.gz"
One code file for API calls
One code file for EDA

## Additional Imports

In [28]:
# Standard
import matplotlib.pyplot as plt
import seaborn as sns

# New Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

# Install tmdbsimple   
!pip install tqdm

#Install tmdbsimple
!pip install tmdbsimple
import tmdbsimple as tmdb

## Define Functions

In [29]:
#get movie ratings for US movies and match them to their movie
def get_movie_with_rating(movie_id):
    
    #Get movie object for current id
    movie = tmdb.Movies(movie_id)
    
    #Save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    #Loop through countries in releases
    for c in releases['countries']:
        
        #if the country == US
        if c['iso_3166_1'] == 'US':
            
            #save the certification(movie rating) with movie
            info['certification'] = c['certification']
    
    return info

In [30]:
#Appends a list of records (new_data) to a json file (filename). 
def write_json(new_data, filename): 
    
    with open(filename,'r+') as file:

        # Load existing data into a dict.
        file_data = json.load(file)
        
        # Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        
        # Sets file's current position at offset.
        file.seek(0)
        
        # convert back to json.
        json.dump(file_data, file)

## Login to TMDB with API Key

In [31]:
import json
with open('/Users/jasontracey/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['API Key', 'API Read Access Token'])

In [32]:
tmdb.API_KEY =  login['API Key']

In [33]:
test = get_movie_with_rating("tt0848228") #put your function name here
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 100.934,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [34]:
## testing our function by looping through a list of ids
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
results = [] 
errors = [] 
for movie_id in test_ids:

    # Track results
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)

    # Track errors    
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28646,PG-13
1,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28646,PG-13
2,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,10348,PG-13


In [35]:
print(f"- Number of errors: {len(errors)}")
errors

- Number of errors: 1


[['tt0115937',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=b25c72a0887d8e5e28603f6accda6e9f')]]

## Verify Data folder creation and verify contents 

In [36]:
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.DS_Store',
 'title_basics.csv.gz',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

In [37]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('/Users/jasontracey/Documents/GitHub/Project-3-Part-1/Data/title_basics.csv.gz')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
86785,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
86786,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
86787,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
86788,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [38]:
YEARS_TO_GET = [2000, 2001]
errors = [ ]

In [39]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json' #JSON file to store results
    file_exists = os.path.isfile(JSON_FILE) # Check if file exists

    # If it does not exist: create it
    if file_exists == False:

        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    
    # saving movie ids to list
    movie_ids = df['tconst'].copy()

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        except Exception as e:
            errors.append([movie_id, e])
            
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                         compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1447 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1568 [00:00<?, ?it/s]

In [40]:
print(f"- Total errors: {len(errors)}")

- Total errors: 444
